In [12]:
!git clone 'https://github.com/Aaru-Siva/UDeblur'
%cd UDeblur/

Cloning into 'UDeblur'...
remote: Enumerating objects: 527, done.
remote: Counting objects: 100% (192/192), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 527 (delta 160), reused 129 (delta 125), pack-reused 335
Receiving objects: 100% (527/527), 16.19 MiB | 24.17 MiB/s, done.
Resolving deltas: 100% (276/276), done.
/content/UDeblur/UDeblur


In [13]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
import gdown
gdown.download('https://drive.google.com/u/0/uc?id=1S0PVRbyTakYY9a82kujgZLbMihfNBLfC&export=download', "/content/UDeblur/experiments/pretrained_models/", quiet=False)

Downloading...
From: https://drive.google.com/u/0/uc?id=1S0PVRbyTakYY9a82kujgZLbMihfNBLfC&export=download
To: /content/UDeblur/experiments/pretrained_models/NAFNet-GoPro-width64.pth
100%|██████████| 272M/272M [00:02<00:00, 100MB/s] 


'/content/UDeblur/experiments/pretrained_models/NAFNet-GoPro-width64.pth'

In [1]:
!pip install streamlit
!pip install pyngrok
!pip install lmdb


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 681.2/681.2 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19897 sha256=7de13b684797874bf73660095529d278815053fa8ba7808f64ac6fff1dd7bbb0
  Stored in directory: c:\users\aaru siva\appdata\local\pip\cache\wheels\99\bb\19\229a2ca20c12e770e05a8044fa290aa80d821755ef353b0065
Successfully built pyngrok



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%%writefile app.py
import streamlit as st
import os
import shutil

from basicsr.models import create_model
from basicsr.utils import img2tensor as _img2tensor, tensor2img, imwrite
from basicsr.utils.options import parse
import numpy as np
import cv2

def imread(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def img2tensor(img, bgr2rgb=False, float32=True):
    img = img.astype(np.float32) / 255.
    return _img2tensor(img, bgr2rgb=bgr2rgb, float32=float32)

def single_image_inference(model, img, save_path):
    model.feed_data(data={'lq': img.unsqueeze(dim=0)})

    if model.opt['val'].get('grids', False):
        model.grids()

    model.test()

    if model.opt['val'].get('grids', False):
        model.grids_inverse()

    visuals = model.get_current_visuals()
    sr_img = tensor2img([visuals['result']])
    imwrite(sr_img, save_path)

def main():
    # Create a folder to store uploaded files
    upload_folder = 'upload/input'
    result_folder = 'upload/output'

    if os.path.isdir(upload_folder):
        shutil.rmtree(upload_folder)
    if os.path.isdir(result_folder):
        shutil.rmtree(result_folder)
    os.makedirs(upload_folder)
    os.makedirs(result_folder)

    # Load the NAFNet model
    opt_path = 'options/test/GoPro/NAFNet-width64.yml' 
    opt = parse(opt_path, is_train=False)
    opt['dist'] = False
    NAFNet = create_model(opt)

    # Create the Streamlit app
    st.title('Image Deblurring App')

    # Add a file uploader to the app
    uploaded_files = st.file_uploader('Upload an image', accept_multiple_files=True)

    if uploaded_files is not None:
      for file in uploaded_files:
          if not file.name.endswith(('.jpg', '.jpeg', '.png')):
              st.error('oops! Please upload image type.')
          else:


    
        # Process the uploaded images
        if uploaded_files:
            for uploaded_file in uploaded_files:
                # Save the uploaded image to the input folder
                with open(os.path.join(upload_folder, uploaded_file.name), 'wb') as f:
                    f.write(uploaded_file.getbuffer())
                
                # Load the input image and process it with the NAFNet model
                input_path = os.path.join(upload_folder, uploaded_file.name)
                img_input = imread(input_path)
                inp = img2tensor(img_input)
                output_path = os.path.join(result_folder, uploaded_file.name)
                single_image_inference(NAFNet, inp, output_path)

            # Display the input and output images
            input_list = sorted(os.listdir(upload_folder))
            output_list = sorted(os.listdir(result_folder))
            for input_path, output_path in zip(input_list, output_list):
                st.image([os.path.join(upload_folder, input_path), os.path.join(result_folder, output_path)],
                        caption=['Input Image', 'Output Image'],
                        width=250)

if __name__ == '__main__':
    main()

Overwriting app.py


In [17]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/UDeblur/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/UDeblur/package.json'
npm WARN UDeblur No description
npm WARN UDeblur No repository field.
npm WARN UDeblur No README data
npm WARN UDeblur No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.474s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [24]:
!streamlit run app.py &>/content/logs.txt &

In [21]:
!npx localtunnel --port 8501

npx: installed 22 in 2.369s
your url is: https://silver-paws-turn.loca.lt
^C
